# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime
import re

In [2]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = '''
SELECT  t.channel_name ,t.activity_name, t.total_describes, t.rejected, t.result, t.order_id,
t.order_number '订单号', t.true_name '姓名', t.user_mobile '手机号', t.product_name '产品名称', t.create_time '下单日期', t.refund_date '预计还款日期', t.reality_refund_date '实际还款日期', 
t.money '应付金额', t.real_pay_money '实付金额', diff_date '逾期天数',t.sort,t.order_method, t.status2, t.activity_id
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
cc.`name` as channel_name
,pa.`name` as activity_name, tprm.order_id
,om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money, tos.sort,
case 	when tos.reality_refund_date is null then DATEDIFF(CURRENT_DATE, tos.refund_date)
			when DATE_FORMAT(tos.reality_refund_date,'%Y-%m-%d')>DATE_FORMAT(tos.refund_date,'%Y-%m-%d') then DATEDIFF(tos.reality_refund_date,tos.refund_date)
			else null END diff_date
,om.order_method,top.total_describes, tprm.activity_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.result') end,'"','') as result 
FROM  db_digua_business.t_order om
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring   tprm on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_risk tori on om.id = tori.order_id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
AND tos.refund_date <= CURRENT_DATE
-- and tprm.activity_id != 335
) t

'''
df_order = query(sql1)
df_order.shape

(120755, 20)

# 提取订单风控数据

# 合并数据

In [5]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(120755)

# 处理日期

In [6]:
df.columns

Index(['channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期',
       '实际还款日期', '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       'activity_id'],
      dtype='object')

In [7]:

df["下单月份"] = df["下单日期"].dt.strftime('%Y-%m')
df['预计还款月份'] = df['预计还款日期'].dt.strftime('%Y-%m')
df.shape


(120755, 22)

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()


拒绝理由
                                                                                     117510
命中云商分低于515强拒(芝麻综合风险等级0豁免)                                                              2105
命中火眸分<550,并且为搜索渠道(自有模型免审豁免)                                                              95
冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                                               80
命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                                              72
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 命中青云分<500,并且为搜索渠道(自有模型免审豁免)                                 69
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 履约历史等级1且为搜索渠道(自有模型免审豁免)                                     35
命中云商分低于515强拒(芝麻综合风险等级0豁免), 冰鉴火眸分<500命中强拒(芝麻综合风险等级0豁免)                                    24
命中火眸分<550,并且为搜索渠道(自有模型免审豁免), 命中青云分<500,并且为搜索渠道(自有模型免审豁免), 履约历史等级1且为搜索渠道(自有模型免审豁免)        24
综合风险等级为3且非免押, 评分等级低于D,命中拒绝                                                               23
命中青云分<500,并且为搜索渠道(自有模型免审豁免), 履约历史等级1且为搜索渠道(自有模型免审豁免)                       

# 处理备注信息

In [9]:

df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

# 渠道归属

In [10]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()


归属渠道
芝麻租物                              79101
搜索渠道                              21898
租机宝                                3708
单人聊天会话中的小程序消息卡片（分享）                3135
未知渠道                               1953
我的小程序入口                            1711
派金花                                1236
扫描二维码                              1060
抖音渠道                                841
其他渠道场景渠道。                           827
付费流量(通过商家数字推广平台，灯火等投放的广告)           756
富士康租机                               547
小程序商家消息（服务提醒）                       448
繁星                                  420
八派信息                                353
智能营销+ocpc                           312
生活号                                 305
小舞直播                                287
勉丫租                                 278
付费灯火                                233
ocpc首页推荐交易                          214
哈银                                  189
付费流量-灯火内测                           142
支付宝社群                               136
系统桌面图标                             

# 订单去重

In [11]:
df.columns

Index(['channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期',
       '实际还款日期', '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       'activity_id', '下单月份', '预计还款月份', '拒绝理由', '来源渠道', '归属渠道'],
      dtype='object')

# 定义状态

In [12]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


# 保留商家的数据

In [13]:
df_contain = df.copy()

# 剔除商家数据

# 去重订单明细导出

In [14]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-10-25'

# 导入服务订单

# 导入出库台账数据

In [15]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# wb = pd.ExcelFile(f_path_ck)  
# df_ck = pd.read_excel(wb, sheet_name="2023")
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(20390, 18)

In [16]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="订单号")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(17311, 48)

In [17]:
dfck.columns

Index(['日期', '订单号', '类目', '型号', '内存', '颜色', '平台套餐', '租金', '期数', '已付金额', '订单归属',
       '出租方', '备注', '快递单号', '供应商', '采购金额', '碎屏保', '串码', '溢价费订单', '单号', '溢价费',
       'channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期',
       '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       'activity_id', '下单月份', '预计还款月份', '拒绝理由', '来源渠道', '归属渠道', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒'],
      dtype='object')

In [18]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


# 到期订单

In [19]:
df.columns

Index(['channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期',
       '实际还款日期', '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       'activity_id', '下单月份', '预计还款月份', '拒绝理由', '来源渠道', '归属渠道', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒'],
      dtype='object')

In [20]:
df = df[df['activity_id']!=335]

In [21]:
Today1 = datetime.now().strftime('%Y-%m-%d')
def order_end(df, month1, month2):
    df4_dq = df[(df['下单月份']==month1)&(df['预计还款月份']==month2)]
    dd_dq_month = df4_dq['订单号'].count()
    # 逾期1+订单
    df_yq_1= df4_dq[(df4_dq['逾期天数']>=1)]
    dd_yq_1 = df_yq_1['订单号'].count()
    # 逾期4+订单
    df_yq_4= df4_dq[(df4_dq['逾期天数']>=4)]
    dd_yq_4 = df_yq_4['订单号'].count()
    # 逾期10+订单
    df_yq_10= df4_dq[(df4_dq['逾期天数']>=10)]
    dd_yq_10 = df_yq_10['订单号'].count()
    # 逾期30+订单
    df_yq_30= df4_dq[(df4_dq['逾期天数']>=30)]
    dd_yq_30 = df_yq_30['订单号'].count()

    # 汇总
    dd_all = pd.DataFrame({
        '下单月份': [month1],
        '本月到期订单': [dd_dq_month],
        '逾期1+订单': [dd_yq_1],
        '逾期4+订单': [dd_yq_4],
        '逾期10+订单': [dd_yq_10],
        '逾期30+订单': [dd_yq_30]
    })
    dd_all['fpd1+逾期率'] = dd_yq_1/dd_dq_month * 100
    dd_all['fpd1+逾期率'] = dd_all['fpd1+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['fpd4+逾期率'] = dd_yq_4/dd_dq_month * 100
    dd_all['fpd4+逾期率'] = dd_all['fpd4+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['fpd10+逾期率'] = dd_yq_10/dd_dq_month * 100
    dd_all['fpd10+逾期率'] = dd_all['fpd10+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['fpd30+逾期率'] = dd_yq_30/dd_dq_month * 100
    dd_all['fpd30+逾期率'] = dd_all['fpd30+逾期率'].apply(lambda x: f'{x: .2f}%')
    
    return df4_dq , dd_all


In [22]:
month1 = '2023-01'
month2 = '2023-02'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-01,37,12,5,4,2,32.43%,13.51%,10.81%,5.41%


In [23]:
month1 = '2023-02'
month2 = '2023-03'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-02,34,5,3,3,1,14.71%,8.82%,8.82%,2.94%


In [24]:
month1 = '2023-03'
month2 = '2023-04'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-03,43,3,3,2,2,6.98%,6.98%,4.65%,4.65%


In [25]:
month1 = '2023-04'
month2 = '2023-05'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-04,101,11,8,4,4,10.89%,7.92%,3.96%,3.96%


In [26]:
month1 = '2023-05'
month2 = '2023-06'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-05,164,16,10,4,2,9.76%,6.10%,2.44%,1.22%


In [27]:
month1 = '2023-06'
month2 = '2023-07'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-06,254,31,15,4,2,12.20%,5.91%,1.57%,0.79%


In [28]:
month1 = '2023-07'
month2 = '2023-08'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-07,453,50,34,23,18,11.04%,7.51%,5.08%,3.97%


In [29]:
month1 = '2023-08'
month2 = '2023-09'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-08,566,97,74,57,42,17.14%,13.07%,10.07%,7.42%


In [30]:
month1 = '2023-09'
month2 = '2023-10'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-09,585,80,50,36,30,13.68%,8.55%,6.15%,5.13%


In [31]:
month1 = '2023-10'
month2 = '2023-11'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-10,752,104,68,51,39,13.83%,9.04%,6.78%,5.19%


In [32]:
month1 = '2023-11'
month2 = '2023-12'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-11,1345,122,73,44,35,9.07%,5.43%,3.27%,2.60%


In [33]:
month1 = '2023-12'
month2 = '2024-01'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2023-12,918,95,48,31,21,10.35%,5.23%,3.38%,2.29%


In [34]:
month1 = '2024-01'
month2 = '2024-02'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-01,667,49,30,16,13,7.35%,4.50%,2.40%,1.95%


In [35]:
month1 = '2024-02'
month2 = '2024-03'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-02,363,20,7,4,3,5.51%,1.93%,1.10%,0.83%


In [36]:
month1 = '2024-03'
month2 = '2024-04'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-03,811,75,38,28,23,9.25%,4.69%,3.45%,2.84%


In [37]:
month1 = '2024-04'
month2 = '2024-05'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-04,1521,131,70,42,33,8.61%,4.60%,2.76%,2.17%


In [38]:
month1 = '2024-05'
month2 = '2024-06'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-05,1214,77,32,27,22,6.34%,2.64%,2.22%,1.81%


In [39]:
month1 = '2024-06'
month2 = '2024-07'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-06,1540,158,82,59,45,10.26%,5.32%,3.83%,2.92%


In [40]:
month1 = '2024-07'
month2 = '2024-08'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-07,1986,188,87,61,44,9.47%,4.38%,3.07%,2.22%


In [41]:
month1 = '2024-08'
month2 = '2024-09'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-08,2137,175,108,78,41,8.19%,5.05%,3.65%,1.92%


In [42]:
month1 = '2024-09'
month2 = '2024-10'
df_dq, dd_all = order_end(df, month1, month2)
# df_dq[['下单日期', '预计还款日期']]
dd_all

,下单月份,本月到期订单,逾期1+订单,逾期4+订单,逾期10+订单,逾期30+订单,fpd1+逾期率,fpd4+逾期率,fpd10+逾期率,fpd30+逾期率
0,2024-09,2040,158,73,34,0,7.75%,3.58%,1.67%,0.00%
